In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


**Requirements**:

1. **LangChain**:
   - LangChain is a framework that integrates various natural language processing (NLP) tools and libraries into a cohesive pipeline for processing and analyzing text data. It provides a modular approach to building NLP applications.

2. **LangChain Community**:
   - LangChain Community extends the functionality of LangChain by offering additional tools, utilities, and integrations contributed by the community. It includes various modules for tasks like graph processing, document loading, vector storage, and more.

3. **OpenAI**:
   - OpenAI is a research organization and AI lab that develops advanced AI models. Their models include language models like GPT (Generative Pre-trained Transformer) and are widely used for natural language understanding and generation tasks.

4. **PyMuPDF**:
   - PyMuPDF is a Python library that provides bindings for the MuPDF PDF rendering library. It allows for reading, writing, and manipulating PDF files, as well as extracting text, images, and other elements from PDF documents.

These tools and libraries serve various purposes, from NLP and AI model integration to PDF document processing and social media interaction.

In [2]:
!pip -q install langchain langchain_community openai tiktoken PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [4]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/drive/MyDrive/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/drive/MyDrive/package.json'
npm WARN MyDrive No description
npm WARN MyDrive No repository field.
npm WARN MyDrive No README data
npm WARN MyDrive No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 9.236s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


* **Streamlit app** : A streamlit app is constructed with an option to upload a PDF document. Its backend is built using the above mentioned technologies in Jupyter Notebook.
* **Uploading file** : File which is of our interest needs to be uploaded. It should be with proper PDF extensions.
* **Text Extraction** : PyMuPDF  extracts the text present in the document by iterating over each page present in it. This processed text is passed on to the OpenAI LLM.
* **OpenAI** : For this project, the LLM model used is  gpt-4-1106-preview. It is provided with the following configurations.
    * **prompt** - Please summarize the following text: "Text: {text} "Summary:"
    * api_key - A paid access token to utilize the functionalities of OpenAI LLMs
* **Summary Generation** : LLM models splits the given text into tokens and summarizes the text using techniques such as Named Entity Recognition.





In [8]:
%%writefile app.py

import streamlit as st
import fitz  # PyMuPDF
from openai import OpenAI

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    document = fitz.open(stream=pdf_path.read())
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

# Function to summarize text using OpenAI API
def summarize_text(text, model="gpt-4-1106-preview"):
    # Replace with your OpenAI API key
    api_key = "sk-*************************************"
    client = OpenAI(api_key=api_key)
    prompt = (
        "Please summarize the following text:\n\n"
        f"Text: {text}\n\n"
        "Summary:"
    )
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# Function to summarize PDF
def summarize_pdf(pdf_file):
    text = extract_text_from_pdf(pdf_file)
    summary = summarize_text(text)
    return summary

# Streamlit app
def main():
    st.title("DocSynth")
    st.markdown(
        """
        This app summarizes text from a PDF file.
        Upload a PDF file and click the button to summarize.
        """
    )

    uploaded_file = st.file_uploader("Upload a PDF file", type="pdf")

    if uploaded_file is not None:
        st.text("PDF file uploaded successfully!")

        # Display some details about the uploaded file
        file_details = {"Filename": uploaded_file.name, "FileSize": uploaded_file.size}
        st.write(file_details)

        # Button to trigger summarization
        if st.button("Summarize"):
            summary = summarize_pdf(uploaded_file)
            st.subheader("Summary")
            st.write(summary)

if __name__ == "__main__":
    main()

Overwriting app.py


In [9]:
!streamlit run app.py &>/content/logs.txt &

In [10]:
!npx localtunnel --port 8501 & curl icanhazip.com

34.125.130.187
npx: installed 22 in 2.839s
your url is: https://bitter-waves-burn.loca.lt
